# 1. Data Frame Transformation #
Example from *Spark: Definitive Guide: Big Data processing Made Simple*, by Mate Zaharia and Bill Chambers - Chapter 2.
Create a Spark session using `pyspark.sql`, create a range of numbers in a DataFrame and then apply two transformations:
+ Count all the even numbers
+ Sort the numbers


In [6]:
! java -version
! python --version

openjdk version "1.8.0_322"
OpenJDK Runtime Environment (build 1.8.0_322-b06)
OpenJDK 64-Bit Server VM (build 25.322-b06, mixed mode)
Python 3.7.9


In [ ]:
! databricks-connect configure -y

In [10]:
from pyspark.sql import SparkSession

try:
    spark = SparkSession \
            .builder \
            .appName("pyspark-nb-1") \
            .master("spark://spark-master:7077") \
            .config("spark.executor.memory", "1024m") \
            .config("spark.eventLog.enabled", "true") \
            .config("spark.eventLog.dir", "file:///opt/workspace/events") \
            .getOrCreate()
    
except Exception as e:
    print(e)

22/04/28 02:22:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Exception in thread "Thread-4" ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/site-packages/py4j/java_gateway.p

An error occurred while calling None.None


java.lang.ExceptionInInitializerError
	at org.apache.spark.api.python.Py4JServer.$anonfun$server$1(Py4JServer.scala:65)
	at java.util.Collections$SingletonList.forEach(Collections.java:4824)
	at py4j.GatewayConnection.run(GatewayConnection.java:250)
	at java.lang.Thread.run(Thread.java:750)
Caused by: java.lang.RuntimeException: Config file /root/.databricks-connect not found. Please run `databricks-connect configure` to accept the end user license agreement and configure Databricks Connect. A copy of the EULA is provided below:

Copyright (2018) Databricks, Inc.

This library (the "Software") may not be used except in connection with the Licensee's use of the Databricks Platform Services pursuant to an Agreement (defined below) between Licensee (defined below) and Databricks, Inc. ("Databricks"). This Software shall be deemed part of the “Subscription Services” under the Agreement, or if the Agreement does not define Subscription Services, then the term in such Agreement that refers t

Generate a range of 1000 numbers

In [ ]:
myrange = spark.range(1000).toDF("number")

  
Find all the even numbers - this is an example of a *Narrow Transformation* (no shuffle) as no data has to be moved between partitions - just a **filter** on a per-partition basis.  Spark uses "lazy evaluation" so nothing is executed at this point:

In [ ]:
divisBy2 = myrange.where("number % 2 = 0")

  
Count all the items in the result-set "divisBy2"; this is an Example of a *Wide Transformation*.  There is an **aggregation** (reduce) that performs several counts on a per-partition basis and then a collect into a final result-set.

In [ ]:
divisBy2.count()

Above is an **action** and causes the previous lines of code to be executed.  Standard types of Actions are:
- view data in console
- collect data to native objects in respective app API language
- write data to output destination
   
Next, sort the *divisBy2* dataframe and take the first 5 numbers from the sorted result   

In [ ]:
divisBy2.sort("number").take(5)

In [ ]:
spark.stop()